# Задание 3: Сравнение производительности CPU vs CUDA

In [1]:
import torch

## 3.1 Подготовка данных

Создайте большие матрицы размеров:
1)  64 x 1024 x 1024
2)  128 x 512 x 512
3) 256 x 256 x 256

Заполните их случайными числами.

In [11]:
# 1) 64x1024x1024
t1 = torch.randn(64,1024,1024)

# 2) 128x512x512
t2 = torch.randn(128,512,512)

# 3) 256x256x256
t3 = torch.randn(256,256,256)

## 3.2 Функция измерения времени

Создайте функцию для измерения времени выполнения операций
- Используйте torch.cuda.Event() для точного измерения на GPU
- Используйте time.time() для измерения на CPU

In [104]:
import time
from typing import Literal

Operand = Literal['@', '+', '*', 'T', 'sum']

def compute_operation(tensor: torch.Tensor, operand: Operand) -> torch.Tensor:
    """
    Performs the specified operation on the tensor.

    Args:
        tensor (torch.Tensor): Input tensor
        operand (Operand): Operation to perform

    Returns:
        torch.Tensor: Result of the operation
    """
    match operand:
        case '@':
            return tensor @ tensor
        case '+':
            return tensor + tensor
        case '*':
            return tensor * tensor
        case 'T':
            return tensor.T
        case 'sum':
            return tensor.sum()
            
def compute_on_gpu(tensor: torch.Tensor, operand: Operand) -> str:
    """
    Measures the execution time of tensor operation on GPU using torch.cuda.Event.

    Args:
        tensor (torch.Tensor): Input tensor
        operand (str): Operation to perform ("@", "+", "*", "T", "sum")
    
    Returns:
        str: Formatted string with GPU runtime in milliseconds

    Raises:
        RuntimeError: If CUDA is not available
    """

    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available on this device.")
 
    tensor = tensor.to('cuda')
    
    # Create CUDA events for precise timing
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    # Warm-up to ensure accurate timing
    _ = compute_operation(tensor, operand)
    torch.cuda.synchronize()

    # Actual timing
    start_event.record()
    _ = compute_operation(tensor, operand)
    end_event.record()

    torch.cuda.synchronize()
    elapsed_ms = start_event.elapsed_time(end_event)

    return f'GPU "{operand}" operation runtime: {elapsed_ms:.1f} ms.'

def compute_on_cpu(tensor: torch.Tensor, operand: Operand) -> str:
    """
    Measures the execution time of tensor operation on CPU using time.perf_counter.

    Args:
        tensor (torch.Tensor): Input tensor
        operand (Operand): Operation to perform ("@", "+", "*", "T", "sum")
    
    Returns:
        str: Formatted string with CPU runtime in milliseconds
    """
    tensor = tensor.to('cpu')
    
    start = time.time()
    _ = compute_operation(tensor, operand)    
    end = time.time()

    elapsed_ms = (end - start) * 1000
    return f'CPU "{operand}" operation runtime: {elapsed_ms:.1f} ms.'

def measure_operation_time(tensor: torch.Tensor, operand: Operand, use_gpu: bool = False) -> str:
    """
    Measures the execution time of tensor operation on CPU or GPU.

    Args:
        tensor (torch.Tensor): Input tensor to perform operation on
        operand (Operand): Operation to perform. Available operations:
            - "@": Matrix multiplication (tensor @ tensor)
            - "+": Element-wise addition (tensor + tensor)
            - "*": Element-wise multiplication (tensor * tensor)  
            - "T": Transpose operation
            - "sum": Sum all elements
        use_gpu (bool, optional): Whether to use GPU for computation. 
                                 Defaults to False (CPU).
    
    Returns:
        str: Formatted string containing device type, operation, and runtime in milliseconds
        
    Raises:
        RuntimeError: If GPU is requested but CUDA is not available
    
    Example:
        >>> tensor = torch.randn(100, 100)
        >>> result = measure_operation_time(tensor, "@", use_gpu=True)
        >>> print(result)
        GPU "@" operation runtime: 1.23 ms
    """
    if use_gpu:
        return compute_on_gpu(tensor, operand)
    else:
        return compute_on_cpu(tensor, operand)


## 3.3 Сравнение операций

Сравните время выполнения следующих операций на CPU и CUDA:

1. Матричное умножение (torch.matmul)
2. Поэлементное сложение
3. Поэлементное умножение
4. Транспонирование
5. Вычисление суммы всех элементов

Для каждой операции:
1. Измерьте время на CPU
2. Измерьте время на GPU (если доступен)
3. Вычислите ускорение (speedup)
4. Выведите результаты в табличном виде

In [131]:
# 1) Матричное умножение

print(f'{t1.shape}: \n {measure_operation_time(t1, "@", use_gpu=True)}\n {measure_operation_time(t1, '@')}\n')
print(f'{t2.shape}: \n {measure_operation_time(t2, "@", use_gpu=True)}\n {measure_operation_time(t2, '@')}\n')
print(f'{t3.shape}: \n {measure_operation_time(t3, "@", use_gpu=True)}\n {measure_operation_time(t3, '@')}\n')

torch.Size([64, 1024, 1024]): 
 GPU "@" operation runtime: 29.3 ms.
 CPU "@" operation runtime: 226.6 ms.

torch.Size([128, 512, 512]): 
 GPU "@" operation runtime: 7.2 ms.
 CPU "@" operation runtime: 58.1 ms.

torch.Size([256, 256, 256]): 
 GPU "@" operation runtime: 1.9 ms.
 CPU "@" operation runtime: 16.1 ms.



In [132]:
# 2) Поэлементное сложение
print(f'{t1.shape}: \n {measure_operation_time(t1, "+", use_gpu=True)}\n {measure_operation_time(t1, '+')}\n')
print(f'{t2.shape}: \n {measure_operation_time(t2, "+", use_gpu=True)}\n {measure_operation_time(t2, '+')}\n')
print(f'{t3.shape}: \n {measure_operation_time(t3, "+", use_gpu=True)}\n {measure_operation_time(t3, '+')}\n')

torch.Size([64, 1024, 1024]): 
 GPU "+" operation runtime: 3.0 ms.
 CPU "+" operation runtime: 24.2 ms.

torch.Size([128, 512, 512]): 
 GPU "+" operation runtime: 1.5 ms.
 CPU "+" operation runtime: 12.4 ms.

torch.Size([256, 256, 256]): 
 GPU "+" operation runtime: 0.8 ms.
 CPU "+" operation runtime: 6.3 ms.



In [134]:
# 3) Поэлементное умножение
print(f'{t1.shape}: \n {measure_operation_time(t1, "*", use_gpu=True)}\n {measure_operation_time(t1, '*')}\n')
print(f'{t2.shape}: \n {measure_operation_time(t2, "*", use_gpu=True)}\n {measure_operation_time(t2, '*')}\n')
print(f'{t3.shape}: \n {measure_operation_time(t3, "*", use_gpu=True)}\n {measure_operation_time(t3, '*')}\n')

torch.Size([64, 1024, 1024]): 
 GPU "*" operation runtime: 3.0 ms.
 CPU "*" operation runtime: 27.8 ms.

torch.Size([128, 512, 512]): 
 GPU "*" operation runtime: 1.5 ms.
 CPU "*" operation runtime: 13.7 ms.

torch.Size([256, 256, 256]): 
 GPU "*" operation runtime: 0.8 ms.
 CPU "*" operation runtime: 6.4 ms.



In [135]:
# 4) Транспонирование
print(f'{t1.shape}: \n {measure_operation_time(t1, "T", use_gpu=True)}\n {measure_operation_time(t1, 'T')}\n')
print(f'{t2.shape}: \n {measure_operation_time(t2, "T", use_gpu=True)}\n {measure_operation_time(t2, 'T')}\n')
print(f'{t3.shape}: \n {measure_operation_time(t3, "T", use_gpu=True)}\n {measure_operation_time(t3, 'T')}\n')

torch.Size([64, 1024, 1024]): 
 GPU "T" operation runtime: 0.0 ms.
 CPU "T" operation runtime: 0.0 ms.

torch.Size([128, 512, 512]): 
 GPU "T" operation runtime: 0.0 ms.
 CPU "T" operation runtime: 0.0 ms.

torch.Size([256, 256, 256]): 
 GPU "T" operation runtime: 0.0 ms.
 CPU "T" operation runtime: 0.0 ms.



In [136]:
# 5) Вычисление суммы всех элементов
print(f'{t1.shape}: \n {measure_operation_time(t1, "sum", use_gpu=True)}\n {measure_operation_time(t1, 'sum')}\n')
print(f'{t2.shape}: \n {measure_operation_time(t2, "sum", use_gpu=True)}\n {measure_operation_time(t2, 'sum')}\n')
print(f'{t3.shape}: \n {measure_operation_time(t3, "sum", use_gpu=True)}\n {measure_operation_time(t3, 'sum')}\n')

torch.Size([64, 1024, 1024]): 
 GPU "sum" operation runtime: 1.4 ms.
 CPU "sum" operation runtime: 6.6 ms.

torch.Size([128, 512, 512]): 
 GPU "sum" operation runtime: 0.7 ms.
 CPU "sum" operation runtime: 3.4 ms.

torch.Size([256, 256, 256]): 
 GPU "sum" operation runtime: 0.4 ms.
 CPU "sum" operation runtime: 1.7 ms.



**Матрица 64x1024x1024:**
| Операция  | CPU (мс) | GPU (мс) | Ускорение |
|------------------------|----------|----------|------------|
| Матричное умножение (@)|   226.6  |   29.3   |   7.7×    |
| Поэлементное cложение (+) |    24.2  |    3.0   |   8.1×    |
| Поэлементное умножение (*) |  27.8  |    3.0   |   9.3×    |
| Транспонирование (T)   |    0.0  |    0.0   |    -   |
| Суммирование (sum)     |    6.6  |    1.4   |    4.7×    |


**Матрица 128x512x512:**
| Операция  | CPU (мс) | GPU (мс) | Ускорение |
|------------------------|----------|----------|------------|
| Матричное умножение (@)|   58.1  |   7.2   |   8.1×    |
| Поэлементное cложение (+) |    12.4  |    1.5   |   8.3×    |
| Поэлементное умножение (*) |  13.7  |    1.5   |   9.1×    |
| Транспонирование (T)   |    0.0  |    0.0   |    -   |
| Суммирование (sum)     |    3.4  |    0.7   |    4.9×    |


**Матрица 256x256x256:**
| Операция  | CPU (мс) | GPU (мс) | Ускорение |
|------------------------|----------|----------|------------|
| Матричное умножение (@)|   16.1  |   1.9   |   8.5×    |
| Поэлементное cложение (+) |    6.3  |    0.8   |   7.9×    |
| Поэлементное умножение (*) |  6.4  |    0.8   |   8.0×    |
| Транспонирование (T)   |    0.0  |    0.0   |    -   |
| Суммирование (sum)     |    1.7  |    0.4   |    4.3×    |

## 3.4 Анализ результатов

Проанализируйте результаты:

1. Какие операции получают наибольшее ускорение на GPU?
2. Почему некоторые операции могут быть медленнее на GPU?
3. Как размер матриц влияет на ускорение?
4. Что происходит при передаче данных между CPU и GPU?

**1. Какие операции получают наибольшее ускорение на GPU?**

Наибольшее ускорение получают операции `@`, `+` и `*`:

- Поэлементное умножение (`*`): 8.0-9.3x
- Поэлементное сложение (`+`): 7.9-8.3x
- Матричное умножение (`@`): 7.7-8.5x

Высокое ускорение обуславливается тем, что все эти поэлементные операции идеально подходят для массивного параллелизма GPU.

Суммирование (`sum`) показывает наименьшее ускорение (4.3-4.9x), т.к. происходит редукция данных (объединение всех элементов в одно значение). 

**2. Почему некоторые операции могут быть медленнее на GPU?**

Каждая операция требует запуска CUDA-ядра, что несёт накладные расходы. Для быстрых операций это может превышать время самих вычислений.

**3. Как размер матриц влияет на ускорение?**

Тенденции по размерам:
| Размер  | Матричное умножение | Поэлементные операции | Суммирование |
|------------------------|----------|----------|------------|
| 64x1024x1024|   7.7x  |   8.1-9.3x   |   4.7×    |
| 128x512x512 |    8.1x  |    8.3-9.1x   |   4.9×    |
| 256x256x256 |  8.5x  |    7.9-8.0x  |   4.3×    |

Ускорение остаётся стабильным для всех размеров.

**4. Что происходит при передаче данных между CPU и GPU?**

Данные копируются через PCIe шину из CPU RAM в GPU VRAM, что в разы медленнее внутренней GPU памяти. Из-за этого накладные расходы на передачу часто превышают время самих вычислений, поэтому GPU эффективен только для больших объемов данных или длительных операций.